In [1]:
#Importing required libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import xlrd
#warning hadle
import warnings# warning filter
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
import uvicorn
import asyncio
import pandas as pd
import numpy as np
from pandas import DataFrame
from fastapi import FastAPI, Request, Form
from fastapi.templating import Jinja2Templates
from datetime import datetime
from fastapi.responses import HTMLResponse
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler

app = FastAPI()
templates = Jinja2Templates(directory='C:\\Users\\Semi\\Desktop')


@app.get("/", response_class=HTMLResponse)
def read_form(request: Request):
    return templates.TemplateResponse("form.html", {"request": request})

@app.post("/predict", response_class=HTMLResponse)
async def predict(
    request: Request,
    CreditScore: float = Form(...),
    FirstTimeHomebuyer: str = Form(...),
    MSA: int = Form(...),
    MIP: int = Form(...),
    OCLTV: float = Form(...),
    DTI: float = Form(...),
    OrigUPB: float = Form(...),
    LTV: float = Form(...),
    OrigInterestRate: float = Form(...),
    PPM: str = Form(...),
    OrigLoanTerm: int = Form(...),
    EverDelinquent: int = Form(...),
    MonthsDelinquent: int = Form(...),
    MonthsInRepayment: int = Form(...),
    LoanPurpose: int = Form(...),
    NumBorrowers: int = Form(...)
):
   
  
    inputs = {
        "CreditScore": CreditScore,
        "FirstTimeHomebuyer": FirstTimeHomebuyer,
        "MSA": MSA,
        "MIP": MIP,
        "OCLTV": OCLTV,
        "DTI": DTI,
        "OrigUPB": OrigUPB,
        "LTV": LTV,
        "OrigInterestRate": OrigInterestRate,
        "PPM": PPM,
        "OrigLoanTerm": OrigLoanTerm,
        "EverDelinquent": EverDelinquent,
        "MonthsDelinquent": MonthsDelinquent,
        "MonthsInRepayment": MonthsInRepayment,
        "LoanPurpose": LoanPurpose,
        "NumBorrowers":NumBorrowers,
                
    }
     # Convert the loan details to a DataFrame
    df = pd.DataFrame([inputs])
    
       
        # Label encode the 'FirstTimeHomebuyer' column
    df['FirstTimeHomebuyer'] = df['FirstTimeHomebuyer'].map({'Yes': 1, 'No': 0})
    df['PPM'] = df['PPM'].map({'Yes': 1, 'No': 0})
    df['LTVRange'] = np.where(df['LTV'] >= 50, 3, np.where(df['LTV'] >= 25, 2, 1))

    # RepaymentRange
    df['RepaymentRange'] = np.where(df['MonthsInRepayment'] >= 192, 5, np.where(df['MonthsInRepayment'] >= 144, 4, 
                                 np.where(df['MonthsInRepayment'] >= 96, 3, np.where(df['MonthsInRepayment'] >= 48, 2, 1))))

    # CreditScoreRange
    df['CreditScoreRange'] = np.where(df['CreditScore'] >= 800, 4, np.where(df['CreditScore'] >= 750, 3, 
                                 np.where(df['CreditScore'] >= 700, 2, 1)))
    
    df['EMI'] = (df['OrigUPB'] * (df['OrigInterestRate']/100) * 
                   ((1 + (df['OrigInterestRate'] / 100)) ** df['OrigLoanTerm']))/(((1 + (df['OrigInterestRate'] / 100)) ** df['OrigLoanTerm']) - 1)
                    
        
    def prepay(dti,income):
        if (dti>40):
            p= income/2
        else:
            p=income*3/4
        return p

    df['Prepay']=np.vectorize(prepay)(df['DTI'],df['EMI']*24)
    df['Prepay']=df['Prepay']-(df['EMI']*24)
    
    
    X = df[['DTI','EMI']]
    
    model=pickle.load(open('linear_regression_pipeline.pkl',"rb"))
    xg_pipeline=pickle.load(open('xg_pipeline.pkl',"rb"))
    
    X_r = df[['CreditScore','FirstTimeHomebuyer','MIP', 'OCLTV','DTI','LTV' ,'OrigInterestRate', 'PPM' ,'OrigLoanTerm' ,'NumBorrowers',
              'MonthsInRepayment','CreditScoreRange','LTVRange','RepaymentRange']]
   

    delinquent = xg_pipeline.predict(X_r)
 
    if delinquent == 1:
        prediction_message = "You are not eligible for the loan"
        prediction = None
    else:
        if X_r.shape[0] == 0:  # Check if X_r DataFrame is empty
            prediction_message = "Data provided does not contain valid samples."
            prediction = None
        else:
            prediction_message = "You are eligible for the loan"
            prediction = round(model.predict(X)[0], 2)


    return templates.TemplateResponse(
        "result.html",
        {"request": request, "prediction_message": prediction_message, "prediction": prediction},
    )
    
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)

ngrok_tunnel

import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2023-07-16T15:54:28+0400 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\Semi\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\Semi\\.ngrok2\\ngrok.yml
INFO:     Started server process [32956]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
t=2023-07-16T16:58:34+0400 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=68ad2083cb54 clientid=94de15bcd893567cab67582bf610e460
t=2023-07-16T16:58:34+0400 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=d84bf715f88b err="session closed"
